# Análisis de sentimientos de películas. 
Se tiene comentarios de películas
en español, que serán clasificadas en las categorías de positivo, negativo. 

# 1. Instalación e importación de librerías.

In [1]:
!pip install inflect
!pip install pandas-profiling==2.7.1

In [2]:
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

import nltk
import nltk.data
import spacy
# Punkt permite separar un texto en frases.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fesar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fesar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fesar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 2. Perfilamiento y entendimiento de los datos

In [3]:
 # Uso de la libreria pandas para la lectura de archivos
data=pd.read_csv('data/MovieReviews.csv', sep=',', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
data_t=data

In [4]:
textos = data_t.copy()
textos['Conteo'] = [len(x) for x in textos['review_es']]
textos['Moda'] = [max(set(x), key=x.count) for x in textos['review_es']]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['review_es']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['review_es']]

# Se realiza un perfilamiento de los datos con la librería pandas profiling
ProfileReport(textos)

Summarize dataset:   0%|          | 0/19 [00:00<?, ?it/s]

ImportError: cannot import name 'ABCIndexClass' from 'pandas.core.dtypes.generic' (C:\Users\fesar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\dtypes\generic.py)

# 3. Preparación de datos

Para poder realizar el pre-procesamiento de los datos, se pasará por tres etapas:
- Limpieza de los datos.
- Tokenización.
- Normalización.

### 3.1 Limpieza de los datos

In [5]:
# Eliminar primera columna sin nombre que es el id
data_t = data_t.drop(data_t.columns[0], axis=1)
data_t.head()

,review_es,sentimiento
0,Si está buscando una película de guerra típica...,positivo
1,Supongo que algunos directores de películas de...,positivo
2,Es difícil contarle más sobre esta película si...,positivo
3,"La película comienza muy lentamente, con el es...",positivo
4,Esta película es verdadera acción en su máxima...,positivo


In [6]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        new_word = re.sub(r'\d+', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

### 3.2 Tokenización

In [12]:
data_t.head()

,review_es,sentimiento
0,Si está buscando una película de guerra típica...,positivo
1,Supongo que algunos directores de películas de...,positivo
2,Es difícil contarle más sobre esta película si...,positivo
3,"La película comienza muy lentamente, con el es...",positivo
4,Esta película es verdadera acción en su máxima...,positivo


In [17]:
data_t['review_es'] = data_t['review_es'].apply(contractions.fix) #Aplica la corrección de las contracciones  
data_t['words'] = data_t['review_es'].apply(word_tokenize)
data_t['words'] = data_t['words'].apply(preprocessing)
data_t.head()

,review_es,sentimiento,words
0,Si está buscando una película de guerra típica...,positivo,"[si, está, buscando, una, película, de, guerra..."
1,Supongo que algunos directores de películas de...,positivo,"[supongo, que, algunos, directores, de, pelícu..."
2,Es difícil contarle más sobre esta película si...,positivo,"[es, difícil, contarle, más, sobre, esta, pelí..."
3,"La película comienza muy lentamente, con el es...",positivo,"[la, película, comienza, muy, lentamente, con,..."
4,Esta película es verdadera acción en su máxima...,positivo,"[esta, película, es, verdadera, acción, en, su..."


###  3.3. Normalización

In [18]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

data_t['words'] = data_t['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
data_t.head()

,review_es,sentimiento,words
0,Si está buscando una película de guerra típica...,positivo,"[si, está, buscando, un, películ, de, guerr, t..."
1,Supongo que algunos directores de películas de...,positivo,"[supongo, que, alguno, direct, de, película, d..."
2,Es difícil contarle más sobre esta película si...,positivo,"[es, difícil, contarl, más, sobr, est, películ..."
3,"La película comienza muy lentamente, con el es...",positivo,"[la, películ, comienz, muy, lenta, con, el, es..."
4,Esta película es verdadera acción en su máxima...,positivo,"[est, películ, es, verdader, acción, en, su, m..."


###  3.4 Selección de campos

In [19]:
data_t['words'] = data_t['words'].apply(lambda x: ' '.join(map(str, x)))
data_t

,review_es,sentimiento,words
0,Si está buscando una película de guerra típica...,positivo,si está buscando un películ de guerr típica es...
1,Supongo que algunos directores de películas de...,positivo,supongo que alguno direct de película de lujo ...
2,Es difícil contarle más sobre esta película si...,positivo,es difícil contarl más sobr est películ sin es...
3,"La película comienza muy lentamente, con el es...",positivo,la películ comienz muy lenta con el estilo de ...
4,Esta película es verdadera acción en su máxima...,positivo,est películ es verdader acción en su máxima ex...
...,...,...,...
4995,"""Criminal decente ordinario"" es triste porque ...",negativo,crimin dec ordinario es trist porqu obvia está...
4996,Savage Island (2003) es una película coja.Es m...,negativo,sav island es un películ coja más como un vide...
4997,Quien escribió el script para esta película no...,negativo,qui escribió el script par est películ no mere...
4998,in a TV-movie 70's kind of way It is one of th...,negativo,in a tvmovie s kind of way it is on of thos mo...


In [20]:
X_data, y_data = data_t['words'],data_t['sentimiento']
y_data = (y_data == 'spam').astype(int)
y_data

0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: sentimiento, Length: 5000, dtype: int32

In [21]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)
print(X_dummy.shape)
X_dummy.toarray()[0]

(5000, 90758)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
count = CountVectorizer()
X_count = count.fit_transform(X_data)
print(X_count.shape)
X_count.toarray()[0]

(5000, 90758)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [23]:
tf_idf = TfidfVectorizer()
X_tf_idf = tf_idf.fit_transform(X_data)

print(X_tf_idf.shape)
X_tf_idf.toarray()[0]

(5000, 90758)


array([0., 0., 0., ..., 0., 0., 0.])

# 4. Creación y Aplicación del modelo